# 微调实战2 - 训练指定格式和字段的输出（2）
**4种方式实现指定格式输出的对比测试**

- 1.经过微调的GPT-3.5-turbo模型
- 2.没有微调，没有通过提示词限定格式的GPT-3.5-turbo模型
- 3.没有微调，通过提示词限定输出格式的GPT-3.5-turbo
- 4.没有微调，通过提示词限定输出格式的GPT-4

## 环境配置

In [ ]:
!pip install load_dotenv ipywidgets pandas
!pip install --upgrade openai

from google.colab import drive
drive.mount('/content/drive')

import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv('/content/drive/MyDrive/Colab/.env')

openai.api_key  = os.environ['OPENAI_API_KEY']

## 定义Chat对话函数

In [ ]:
from openai import OpenAI
client = OpenAI()
def get_completion(messages, model="gpt-3.5-turbo"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,

        temperature=0 # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pandas as pd

prompt = "As a content moderator, your role involves assessing user feedback. "
prompt_expand = """
As a content moderator, your role involves receiving user feedback.
Your primary task is to extract essential information from this feedback and structure it into a JSON format,
as follows:
{aspect: the property under evaluation, sentiment: positive/negative/neutral, and category: the category label}.
Only a single JSON entry is required.
All the information within the JSON is expected to be in English.
"""
user_content = "这个酒店环境还行，服务一般"
# 创建多个对象，每个对象包含不同的messages和model
objects = [
    {
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_content},
        ],
        "model": "ft:gpt-3.5-turbo-0613:tiaotiaotang-creative-and-tech-ldt::8DPsEN5Y",
    },
    {
        "messages": [
            {"role": "system", "content": prompt},
            {"role": "user", "content": user_content},
        ],
        "model": "gpt-3.5-turbo",
    },
    {
        "messages": [
            {"role": "system", "content": prompt_expand},
            {"role": "user", "content":user_content},
        ],
        "model": "gpt-3.5-turbo",
    },
    {
        "messages": [
            {"role": "system", "content": prompt_expand},
            {"role": "user", "content":user_content},
        ],
        "model": "gpt-4",
    }
]

# 创建DataFrame
data = []
for obj in objects:
    response = get_completion(obj["messages"], obj["model"])
    data.append({"Model": obj["model"], "Prompt": obj["messages"][0]["content"], "Response": response})

df = pd.DataFrame(data)

# 创建表格
table = widgets.Output()
with table:
    display(df.style.set_properties(**{'text-align': 'left'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'left')]}]))

# 显示表格
display(table)


Output()

## 更新：最新出的1106版本可以输出json,但是要有以下注意事项
### 当使用JSON模式时，您必须通过系统消息或用户消息亲自指示模型生成JSON。如果没有这样做，模型可能会生成一系列不断的空白，直到生成达到令牌限制，导致一个持续运行且看似“卡住”的请求。还请注意，如果`finish_reason="length"`，表示生成超过了`max_tokens`或对话超过了最大上下文长度，消息内容可能会被部分截断。

In [ ]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  messages=[
    {"role": "system", "content": prompt_expand},
    {"role": "user", "content": user_content}
  ],
  response_format={
     "type":"json_object"
  }

)
print(completion.choices[0].message.content)

{
  "aspect": "environment",
  "sentiment": "neutral",
  "category": "accommodation"
}
